In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign, expand
from IPython.display import display, HTML, Math

In [2]:
def large(expr):
  latex_code = f"\\Large {{ {latex(expr)} }}"
  display(Math(latex_code))

In [3]:
delta_b_negative = False
discrete = False
simp = True

In [4]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'l', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, l_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, l_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

in the contract, where Delta_b is negative:

-Delta_b * s_s <= Delta_s * s_b

implying for positive Delta_b:

Delta_b * s_s <= Delta_s * s_b

which maximizes value for the trader if:

In [5]:
if delta_b_negative:
  a0_eq_Delta = Eq(-Delta_b * s_s, Delta_s * s_b)
else:
  a0_eq_Delta = Eq(Delta_b * s_s, Delta_s * s_b)
a0_eq_Delta

Eq(Delta_b*s_s, Delta_s*s_b)

In [6]:
if delta_b_negative:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b + Delta_b) * s_s + (l_s + Delta_s) * s_b)
else:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b - Delta_b) * s_s + (l_s + Delta_s) * s_b)
a0_eq_Value

Eq(l_b*s_s + l_s*s_b, s_b*(Delta_s + l_s) + s_s*(-Delta_b + l_b))

In [7]:
assert(solveset(a0_eq_Value, Delta_b).simplify() == solveset(a0_eq_Delta, Delta_b))
assert(solveset(a0_eq_Value, Delta_s).simplify() == solveset(a0_eq_Delta, Delta_s))
a0_eq = a0_eq_Delta

In [8]:
# this seems to be derived from spotByDelta
def deltaBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = (s - l * w) / w # at this point, this value is negative for buying and positive for selling
  # rounding towards zero in all cases, as we are interested in the maximum delta-capacity of a given spot price
  if asset == 'b':
    if delta_b_negative:
      if discrete:
        f = ceiling(f)
    else:
      if discrete:
        f = floor(-f)
      else:
        f = -f

  else:
    if discrete:
      f = floor(f)

  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaBySpot['b']))
display(Eq(Delta_s, deltaBySpot['s']))

Eq(Delta_b, l_b - s_b/w_b)

Eq(Delta_s, -l_s + s_s/w_s)

In [9]:
solveset(Eq(Delta_b, deltaBySpot['b']), s_b)

{-w_b*(Delta_b - l_b)}

In [10]:
solveset(Eq(Delta_s, deltaBySpot['s']), s_s)

{w_s*(Delta_s + l_s)}

### logic behind spotByDelta

- we need to pick a spot price that is worse than the amm price = w * l
- for uninverted prices, worse means larger for buying and lower for selling
- since prices are inverted, worse means smaller for buying and larger for selling
- this must hold true after the trade as well, so it must also be larger than w * (l + Delta) for selling and smaller than w * (l - Delta) for buying 
- (assuming Delta_b is represented positively, otherwise it's larger (selling) resp. smaller (buying) than w * (l + Delta) for both)

In [11]:
def spotByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  if (not delta_b_negative) and asset == 'b':
    f = w * (l - Delta)
  else:
    f = w * (l + Delta)
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(s_b, spotByDelta['b']))
display(Eq(s_s, spotByDelta['s']))

Eq(s_b, w_b*(-Delta_b + l_b))

Eq(s_s, w_s*(Delta_s + l_s))

In [12]:
assert(Eq(spotByDelta['b'].subs({Delta_b: deltaBySpot['b']}), s_b))
assert(Eq(spotByDelta['s'].subs({Delta_s: deltaBySpot['s']}), s_s))
if discrete:
  display(deltaBySpot['b'].subs({s_b: spotByDelta['b']}))
  display(deltaBySpot['s'].subs({s_s: spotByDelta['s']}))
else:
  assert(Eq(deltaBySpot['b'].subs({s_b: spotByDelta['b']}), Delta_b))
  assert(Eq(deltaBySpot['s'].subs({s_s: spotByDelta['s']}), Delta_s))

In [13]:
a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}).simplify()

Eq(Delta_b*w_s*(Delta_s + l_s), Delta_s*w_b*(-Delta_b + l_b))

In [14]:
def deltaByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  # if discrete:
  #   if asset == 'b' and delta_b_negative:
  #     domain = S.Integers
  #   else:
  #     domain = S.Naturals
  # else:
  #   domain = S.Reals
  # above takes more than 11 minutes, skipping for now
  f = solveset(a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}), Delta)#, domain=domain)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if discrete:
    if asset == 'b':
      f = floor(f)
    else:
      f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaByDelta = {asset: deltaByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaByDelta['b']))
display(Eq(Delta_s, deltaByDelta['s']))

Eq(Delta_b, Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))

Eq(Delta_s, -Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))

### note:

 - the above is undefined/negative (which reduces to undefined considering the meaning) for some values.
 - when we represent Delta_b negatively, this issue appears to vanish, but actually doesn't when pondering the sign of Delta_b

In [15]:
denominator_Delta_b = deltaByDelta['b'].args[-1].args[0]
display(denominator_Delta_b)
ndef_Delta_s = solveset(denominator_Delta_b, Delta_s)
assert(len(ndef_Delta_s) == 1)
ndef_Delta_s = list(ndef_Delta_s)[0]
ndef_Delta_s

Delta_s*w_b + Delta_s*w_s + l_s*w_s

-l_s*w_s/(w_b + w_s)

In [16]:
denominator_Delta_s = deltaByDelta['s'].args[-1].args[0]
display(denominator_Delta_s)
ndef_Delta_b = solveset(denominator_Delta_s, Delta_b)
assert(len(ndef_Delta_b) == 1)
ndef_Delta_b = list(ndef_Delta_b)[0]
ndef_Delta_b

Delta_b*w_b + Delta_b*w_s - l_b*w_b

l_b*w_b/(w_b + w_s)

In [17]:
display(solveset(a0_eq, Delta_s))
display(solveset(a0_eq, Delta_b))

{Delta_b*s_s/s_b}

{Delta_s*s_b/s_s}

given that we can lower s_b arbitrarily (up to a lower bound of 1) and increase s_s arbitrarily, we should always be able to get a positive Delta_s


given that we can get some negative Delta_s, does this imply an exploit?

- in the onchain-code, the swap prices can never be negative, as they are computed there from exponents
- likewise, if Delta_b is negative (as it should be there), -Delta_b * s_s is positive
- -> negative Delta_s would violate -Delta_b * s_s <= Delta_s * s_b (for negative Delta_b)

-> no exploit onchain (except potentially negating both deltas, which we examine further below)

### how do we adjust the equations for the spot prices accordingly, such that the user still get's the best effective price?
And after we found an equation, how does it reconcile with the one we got before?


In [18]:
lowest_spot = (s_s / s_b).subs({s_s: spotByDelta['s'], s_b: spotByDelta['b']})#.simplify()
lowest_spot

w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

lowest_spot: 
- best (lowest) uninverted spot price the user can get (with highest inverted buying price and lowest inverted selling price)
- evidently always positive

In [19]:
Ge(Delta_s, (Delta_b * lowest_spot))#.simplify()

Delta_s >= Delta_b*w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

above:
- as Delta_b is positive (in the respective representation) and spot is positive, Delta_s should always be positive
- this however is just a rearrangement of what we already have, resulting in the same paradox

### adding extra price multiplier

given that for Delta_b -> l_b * w_b / (w_b + w_s) Delta_s goes towards infinity in the equation derived from choosing best possible prices, we add an additional multiplier >= 1 to reflect worsening of the pair spot price for situations where Delta_b >= l_b * w_b / (w_b + w_s). This can then be translated in an arbitrary combination of worsening of s_b and s_s.

In [20]:
ndef_Delta_b

l_b*w_b/(w_b + w_s)

In [21]:
m = symbols('m', nonnegative=True, real=True)
m_Delta_s = Delta_b * lowest_spot * (1 + m)
m_Delta_s

Delta_b*w_s*(Delta_s + l_s)*(m + 1)/(w_b*(-Delta_b + l_b))

In [22]:
m_DeltaByDelta_b = solveset(Eq(m_Delta_s, Delta_s), Delta_b)
display(m_DeltaByDelta_b)
m_DeltaByDelta_b = m_DeltaByDelta_b.args[0].args[0]
m_DeltaByDelta_b

Complement({Delta_s*l_b*w_b/(Delta_s*m*w_s + Delta_s*w_b + Delta_s*w_s + l_s*m*w_s + l_s*w_s)}, {l_b})

Delta_s*l_b*w_b/(Delta_s*m*w_s + Delta_s*w_b + Delta_s*w_s + l_s*m*w_s + l_s*w_s)

In [23]:
m_DeltaByDelta_s = solveset(Eq(m_Delta_s, Delta_s), Delta_s)
display(m_DeltaByDelta_s)
m_DeltaByDelta_s = m_DeltaByDelta_s.args[0]
m_DeltaByDelta_s

{-Delta_b*l_s*w_s*(m + 1)/(Delta_b*m*w_s + Delta_b*w_b + Delta_b*w_s - l_b*w_b)}

-Delta_b*l_s*w_s*(m + 1)/(Delta_b*m*w_s + Delta_b*w_b + Delta_b*w_s - l_b*w_b)

In [24]:
m_DeltaByDelta_s.subs({Delta_b: ndef_Delta_b})

-l_b*l_s*w_b*w_s*(m + 1)/((w_b + w_s)*(l_b*m*w_b*w_s/(w_b + w_s) + l_b*w_b**2/(w_b + w_s) + l_b*w_b*w_s/(w_b + w_s) - l_b*w_b))

In [25]:
m_DeltaByDelta_s.subs({Delta_b: ndef_Delta_b}).simplify()

-l_s - l_s/m

this is always negative - why didn't it work?

-> we are at least making the mistake of effectively increasing Delta_s on the lhs, without adjusting s_s accordingly

(additive also didn't work)

In [26]:
Ge(Delta_s, (Delta_b * lowest_spot)).subs({Delta_b: ndef_Delta_b})#.simplify()

Delta_s >= l_b*w_s*(Delta_s + l_s)/((w_b + w_s)*(-l_b*w_b/(w_b + w_s) + l_b))

In [27]:
(Delta_b * lowest_spot).subs({Delta_b: ndef_Delta_b}).simplify()

Delta_s + l_s

In [28]:
(Delta_b * lowest_spot * (m+1)).subs({Delta_b: ndef_Delta_b}).simplify()

(Delta_s + l_s)*(m + 1)

we can see that the undefined Delta_b results in a situation where Delta_s >= smth gt Delta_s

let's try correcting s_s first:

In [29]:
display(m_Delta_s)
m_Delta_s_2 = m_Delta_s.subs({Delta_s: Delta_s / (1 + m)}).simplify()
m_Delta_s_2

Delta_b*w_s*(Delta_s + l_s)*(m + 1)/(w_b*(-Delta_b + l_b))

-Delta_b*w_s*(Delta_s + l_s*(m + 1))/(w_b*(Delta_b - l_b))

In [30]:
m_DeltaByDelta_s_2 = solveset(Eq(m_Delta_s_2, Delta_s), Delta_s)
display(m_DeltaByDelta_s_2)
m_DeltaByDelta_s_2 = m_DeltaByDelta_s_2.args[0]
m_DeltaByDelta_s_2

{-Delta_b*l_s*w_s*(m + 1)/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)}

-Delta_b*l_s*w_s*(m + 1)/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)

In [31]:
m_DeltaByDelta_s_2.subs({Delta_b: ndef_Delta_b}).simplify()

zoo

### we conclude 

that the unsolveablity of the issue boils down to the simple fact that:
- as Delta_b approaches l_b * w_b / (w_b + w_s), Delta_s approaches infinity
- this means the effective price Delta_s / Delta_b approaches infinity
- if we were able to remedy that, we would be able to reduce the effective price at least at some points, which should not be allowed

-> let's just note the additional constraint

TODO: Add this check to the offchain/frontend such that the pool creator knows when some liquidity can never be bought. This might however require some additional analysis for the multi-asset-case. Also note that each swap reduces l_b, so maybe this is a non-issue after all



(below is just some check that fits nowhere really)

In [32]:
solveset(Eq(Delta_s, deltaByDelta['s']), Delta_b)

Complement({Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s)}, {l_b*w_b/(w_b + w_s)})

In [33]:
solveset(Eq(Delta_b, deltaByDelta['b']), Delta_s)

Complement({-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)}, {-l_s*w_s/(w_b + w_s)})

### what happens if an attacker simply removes the "selling" asset and adds the "buying" one instead of the other way around?

- onchain equation: -Delta_b * s_s <= Delta_s * s_b
- representing Delta_b positively: Delta_b * s_s <= Delta_s * s_b
- => effectivePrice = Delta_s / Delta_b >= s_s / s_b = spotPrice
- spotPrice conditions:
  - prior: s_b <= w_b * l_b, s_s >= w_s * l_s 
    - => spotPrice >= w_s * l_s / (w_b * l_b)
  - posterior: s_b <= w_b * (l_b - Delta_b), s_s >= w_s * (l_s + Delta_s) 
    - => spotPrice >= w_s * (l_s + Delta_s) / (w_b * (l_b - Delta_b))

if Delta_b is positive and Delta_s is negative:
- the equation is not violated per se
- the effective price stays the same as the negations cancel out
- however, the condition for the posterior spotPrice changes:
  - s_b <= w_b * (l_b + Delta_b), s_s >= w_s * (l_s - Delta_s) 
    - => spotPrice >= w_s * (l_s - Delta_s) / (w_b * (l_b + Delta_b))
      - --> this constitutes an exploit!

### fix
- require correct signs for balance-changes
- we can also drop the pre-swap price requirement, as it's subsumed by the post-swap one


## worst case rounding analysis

- rounding the Deltas cannot do worse than increasing Delta_s by 1 and decreasing Delta_b by 1
- spotByDelta has no divisions and is therefore unaffected by rounding
- we are not accounting yet for the rounding effect of having to express spots via exponents, not here or above (TODO, TODO)

### in this copy of the notebook, we instead maximize the inverted effective price, in hopes of getting nicer equations

In [34]:
eff_best = Delta_b / Delta_s # inverted effective price we want to maximize
display(eff_best)
eff_worst = (Delta_b - 1) / (Delta_s + 1) # noninclusive worst case resulting from rounding Deltas
eff_worst

Delta_b/Delta_s

(Delta_b - 1)/(Delta_s + 1)

In [35]:
large(eff_best.subs({Delta_b: deltaByDelta['b']}).simplify())
large(eff_best.subs({Delta_s: deltaByDelta['s']}).simplify())

<IPython.core.display.Math object>

<IPython.core.display.Math object>

-> simply maximized by minimizing Deltas

In [36]:
eff_worst_Delta_b = eff_worst.subs({Delta_s: deltaByDelta['s']})
large(eff_worst_Delta_b)
eff_worst_Delta_b = eff_worst_Delta_b.simplify()
large(eff_worst_Delta_b)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [37]:
large(expand(eff_worst_Delta_b).simplify())

<IPython.core.display.Math object>

In [42]:
numerator = -Delta_b**2 * (w_b + w_s) + Delta_b * (l_b * w_b + w_b + w_s) - l_b * w_b
denominator = Delta_b * (l_s * w_s - w_b - w_s) + l_b * w_b
f = numerator / denominator
large(f)
Eq(f, eff_worst_Delta_b).simplify()

<IPython.core.display.Math object>

True

In [44]:
a, b, c, d_pos = symbols('a b c d_pos', positive=True, integer=True)
d_neg = symbols('d_neg', negative=True, integer=True)
d = symbols('d', integer=True)
f0 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, l_s * w_s - w_b - w_s: d})
display(f0)
f1 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, l_s * w_s - w_b - w_s: d_pos})
display(f1)
f2 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, l_s * w_s - w_b - w_s: d_neg})
display(f2)
f3 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, l_s * w_s - w_b - w_s: 0})
display(f3)

(-Delta_b**2*a + Delta_b*b - c)/(Delta_b*d + c)

(-Delta_b**2*a + Delta_b*b - c)/(Delta_b*d_pos + c)

(-Delta_b**2*a + Delta_b*b - c)/(Delta_b*d_neg + c)

(-Delta_b**2*a + Delta_b*b - c)/c

In [45]:
d_f0 = f0.diff(Delta_b).simplify()
display(d_f0)
d_f1 = f1.diff(Delta_b).simplify()
display(d_f1)
d_f2 = f2.diff(Delta_b).simplify()
display(d_f2)
d_f3 = f3.diff(Delta_b).simplify()
display(d_f3)

(d*(Delta_b**2*a - Delta_b*b + c) + (-2*Delta_b*a + b)*(Delta_b*d + c))/(Delta_b*d + c)**2

(d_pos*(Delta_b**2*a - Delta_b*b + c) + (-2*Delta_b*a + b)*(Delta_b*d_pos + c))/(Delta_b*d_pos + c)**2

(d_neg*(Delta_b**2*a - Delta_b*b + c) + (-2*Delta_b*a + b)*(Delta_b*d_neg + c))/(Delta_b*d_neg + c)**2

(-2*Delta_b*a + b)/c

In [46]:
sols_f0 = solveset(d_f0, Delta_b)
display(sols_f0)
sols_f1 = solveset(d_f1, Delta_b)
display(sols_f1)
sols_f2 = solveset(d_f2, Delta_b)
display(sols_f2)
sols_f3 = solveset(d_f3, Delta_b)
display(sols_f3)

Complement({-c/d - sqrt(c)*sqrt(a*c + b*d + d**2)/(sqrt(a)*d), -c/d + sqrt(c)*sqrt(a*c + b*d + d**2)/(sqrt(a)*d)}, {-c/d})

{-c/d_pos - sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos), -c/d_pos + sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos)}

Complement({-c/d_neg - sqrt(c)*sqrt(a*c + b*d_neg + d_neg**2)/(sqrt(a)*d_neg), -c/d_neg + sqrt(c)*sqrt(a*c + b*d_neg + d_neg**2)/(sqrt(a)*d_neg)}, {-c/d_neg})

{b/(2*a)}

In [50]:
sol0_1 = sols_f0.args[0].args[0]
# display(sol0_1)
sol0_2 = sols_f0.args[0].args[1]
# display(sol0_2)
sol1_1 = sols_f1.args[0].args[0]
display(sol1_1)
print("d negative -> solution negative -> invalid")
sol1_2 = sols_f1.args[0].args[1]
display(sol1_2)
print("everything positive -> solution positive -> valid")
sol2_1 = sols_f2.args[0].args[0]
display(sol2_1)
print("first summand positive, second negative or irrational -> sometimes valid")
sol2_2 = sols_f2.args[0].args[1]
display(sol2_2)
print("both summands positive, second sometimes irrational -> sometimes valid")
sol3 = sols_f3.args[0]
display(sol3)
print("a, b positive -> solution positive -> valid")

-c/d_pos

d negative -> solution negative -> invalid


sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos)

everything positive -> solution positive -> valid


-c/d_neg + sqrt(c)*sqrt(a*c + b*d_neg + d_neg**2)/(sqrt(a)*d_neg)

first summand positive, second negative or irrational -> sometimes valid


-c/d_neg - sqrt(c)*sqrt(a*c + b*d_neg + d_neg**2)/(sqrt(a)*d_neg)

both summands positive, second sometimes irrational -> sometimes valid


b/(2*a)

a, b positive -> solution positive -> valid


In [ ]:
sol2_2

In [ ]:
raise Exception("stop here")

In [ ]:
dd_f0 = d_f0.diff(Delta_s).simplify()
display(dd_f0)
# dd_f1 = d_f1.diff(Delta_s).simplify()
# display(dd_f1)
# dd_f2 = d_f2.diff(Delta_s).simplify()
# display(dd_f2)

In [ ]:
dd_sol1 = dd_f0.subs({Delta_s: sol1}).simplify()
display(dd_sol1)
if Eq(dd_sol1, 0) == False: print("-> extremum")
dd_sol2 = dd_f0.subs({Delta_s: sol2}).simplify()
display(dd_sol2)
if Eq(dd_sol2, 0) == False: print("-> extremum")

In [ ]:
dd_sol1 = dd_sol1.subs({a: w_b + w_s, b: l_s * w_s + w_b + w_s, c: l_s * w_s, d: l_b * w_b - w_b - w_s}).simplify()
display(dd_sol1)
if Eq(dd_sol1, 0) == False: print("-> extremum")
dd_sol2 = dd_sol2.subs({a: w_b + w_s, b: l_s * w_s + w_b + w_s, c: l_s * w_s, d: l_b * w_b - w_b - w_s}).simplify()
display(dd_sol2)
if Eq(dd_sol2, 0) == False: print("-> extremum")

In [ ]:
raise Exception("stop here")

In [ ]:
d_eff_worst_Delta_s = eff_worst_Delta_s.diff(Delta_s)
large(d_eff_worst_Delta_s)
d_eff_worst_Delta_s = d_eff_worst_Delta_s.simplify()
large(d_eff_worst_Delta_s)

In [ ]:
sols_s = solveset(d_eff_worst_Delta_s, Delta_s)
large(sols_s)

-> what to make of that excluded Delta_s?

In [ ]:
raise Exception("stop here")

In [ ]:
eff_worst_Delta_b = eff_worst.subs({Delta_s: deltaByDelta['s']})
large(eff_worst_Delta_b)
eff_worst_Delta_b = eff_worst_Delta_b.simplify()
large(eff_worst_Delta_b)

In [ ]:
large(expand(eff_worst_Delta_b).simplify())

In [ ]:
# numerator = Delta_b * (- l_s * w_s + w_b + w_s) - l_b * w_b
# denominator = Delta_b**2 * (w_b + w_s) - Delta_b * (l_b * w_b + w_b + w_s) + l_b * w_b
# f = numerator / denominator
# large(f)
# Eq(f, eff_worst_Delta_b).simplify()

In [ ]:
# a, b, c, d_pos = symbols('a b c d_pos', positive=True, integer=True)
# d_neg = symbols('d_neg', negative=True, integer=True)
# f1 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, -l_s * w_s + w_b + w_s: d_pos})
# display(f1)
# f2 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, -l_s * w_s + w_b + w_s: d_neg})
# display(f2)
# f3 = f.subs({w_b + w_s: a, l_b * w_b + w_b + w_s: b, l_b * w_b: c, -l_s * w_s + w_b + w_s: 0})
# display(f3)

In [ ]:
# d_f1 = f1.diff(Delta_b).simplify()
# display(d_f1)
# d_f2 = f2.diff(Delta_b).simplify()
# display(d_f2)
# d_f3 = f3.diff(Delta_b).simplify()
# display(d_f3)

In [ ]:
# sols_f1 = solveset(d_f1, Delta_b)
# display(sols_f1)
# sols_f2 = solveset(d_f2, Delta_b)
# display(sols_f2)
# sols_f3 = solveset(d_f3, Delta_b)
# display(sols_f3)

In [ ]:
d_eff_worst_Delta_b = eff_worst_Delta_b.diff(Delta_b)
large(d_eff_worst_Delta_b)
d_eff_worst_Delta_b = d_eff_worst_Delta_b.simplify()
large(d_eff_worst_Delta_b)

In [ ]:
sols_b = solveset(d_eff_worst_Delta_b, Delta_b)
large(sols_b)

-> we already excluded the second excluded Delta_b, but what to make of the excluded Delta_b = 1?

In [ ]:
large(sols_s.args[0].args[0])#.simplify())
large(sols_s.args[0].args[1])#.simplify())
large(sols_b.args[0].args[0])#.simplify())
large(sols_b.args[0].args[1])#.simplify())

### -> what to do with this?

- 

In [ ]:
raise Exception("stop here")

(below: some variant to deltaByDelta: spotBySpot)

In [ ]:
def spotBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = solveset(a0_eq.subs({Delta_b: deltaBySpot['b'], Delta_s: deltaBySpot['s']}), s)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if simp:
    f = simplify(f)
  return f

spotBySpot = {asset: spotBySpot_(asset, *all_symbols[asset]) for asset in assets}
spotBySpot['b']

In [ ]:
spotBySpot['s']

all four solutions are the same irrespective of delta_b_negative (continuous case, and not accounting for flipped indices, of course)


In [ ]:
denominator_s_s = spotBySpot['s'].args[-1].args[0]
display(denominator_s_s)
ndef_s_b = solveset(denominator_s_s, s_b)
assert(len(ndef_s_b) == 1)
ndef_s_b = ndef_s_b.args[0]
ndef_s_b

In [ ]:
denominator_s_b = spotBySpot['b'].args[-1].args[0]
display(denominator_s_b)
ndef_s_s = solveset(denominator_s_b, s_s)
assert(len(ndef_s_s) == 1)
ndef_s_s = ndef_s_s.args[0]
ndef_s_s

In [ ]:
display(ndef_Delta_b)
display(ndef_Delta_s)

In [ ]:
display(ndef_Delta_b * -w_s)
display(ndef_Delta_s * -w_b)

- the implied limits on spot prices look similar to those on Delta_b
- this similarity is encapsulated by negation and multiplication by the other asset's weight, which is odd, but reminds us a bit of the amm price calculation

what happens if we use those invalid spot prices in our other equations? Do they break or not?

In [ ]:
ndef_s_s = solveset(spotBySpot['b'].args[-1].args[0], s_s)
assert(len(ndef_s_s) == 1)
ndef_s_s = ndef_s_s.args[0]
ndef_s_s

In [ ]:
a0_ndef_spots = a0_eq.subs({s_b: ndef_s_b, s_s: ndef_s_s})
a0_ndef_spots

In [ ]:
solveset(a0_ndef_spots, Delta_b)

In [ ]:
solveset(a0_eq, Delta_s)

In [ ]:
solveset(a0_ndef_spots, Delta_s)